# Generate ATP Synthase with Proton Pump SBML files
Will try to incorporate it instead of using SBML 

---

In [1]:
from biocrnpyler import *
#from biocrnpyler.chemical_reaction_network import Species, Reaction, ComplexSpecies, Multimer
import numpy as np
import pylab as plt 
%matplotlib inline

import bokeh.io
import bokeh.plotting
bokeh.io.output_notebook()

import colorcet

Loading BokehJS ...

ATP synthase DNA --> ATP synthase RNA <br>
ATP synthase RNA --> ATP synthase protein (folding) <br>
ATP synthase protein --> ATP synthase membrane integration <br>
<br> 
ATP synthesis through ATP synthase protein <br>
ATP hydrolysis through ATP synthase protein <br>
Proton transport through ATP synthase protein <br>
<br>
Proton pump DNA --> Proton pump RNA <br>
Proton pump RNA --> Proton pump protein (folding) <br>
Proton pump protein --> Proton pump membrane integration <br>
<br>
Proton transport through proton pump

**SBML Planning** <br>
- One sbml file for ATP synthase membrane integration
- One sbml file for ATP synthase synthesis/hydrolysis/proton transport 
- One sbml file for proton pump membrane integration
- One sbml file for proton pump proton transport

## SBML for ATP transcription and translation
---

In [2]:

#promoter is the name of the promoter or a Promoter component
#RBS is the name of the RBS or an RBS Component
#   RBS = None means there will be no translation
#By default (if transcript and protein are None), the transgript and protein will have the same name as an assmebly
#   Users can also assign Species or String names to transcript and protein
G = DNAassembly("atpsyn", promoter = "atpsyn_promoter", rbs = "atpsyn_rbs", transcript = None, protein = None)
M3 = TxTlDilutionMixture("liposome", components = [G], parameter_file = "default_parameters.txt")
CRN3 = M3.compile_crn()


print(repr(M3),"\n", CRN3.pretty_print(show_attributes = True, show_material = True, show_rates = True),"\n\n")


TxTlDilutionMixture: liposome
Components = [
	DNAassembly: atpsyn
	Protein: RNAP
	Protein: Ribo
	Protein: RNAase
	DNAassembly: cellular_processes ]
Mechanisms = {
	transcription:transcription_mm
	translation:translation_mm
	catalysis:michalis_menten
	binding:one_step_binding }
Global Mechanisms = {
	rna_degredation:rna_degredation_mm
	dilution:global_degredation_via_dilution } 
 Species (17) = {0. dna[atpsyn] init_conc = 0, 1. protein[RNAP(machinery)] init_conc = 0, 2. rna[atpsyn] init_conc = 0, 3. complex[dna[atpsyn]:protein[RNAP]] init_conc = 0, 4. protein[Ribo(machinery)] init_conc = 0, 5. protein[atpsyn] init_conc = 0, 6. complex[protein[Ribo]:rna[atpsyn]] init_conc = 0, 7. protein[RNAase(machinery)] init_conc = 0, 8. dna[cellular_processes] init_conc = 0, 9. rna[cellular_processes] init_conc = 0, 10. complex[dna[cellular_processes]:protein[RNAP]] init_conc = 0, 11. protein[cellular_processes] init_conc = 0, 12. complex[protein[Ribo]:rna[cellular_processes]] init_conc = 0, 13. comp

In [3]:
# Write SBML file
CRN3.write_sbml_file('atpsynthase_tx_tl.xml')

True

In [4]:
timepoints = np.linspace(0,10000,100)
x0 = {'dna_atpsyn':10, 'protein_RNAP_machinery':10, 'protein_Ribo_machinery':10,
     'protein_RNAase_machinery':10}
R = CRN3.simulate_with_bioscrape(timepoints, initial_condition_dict = x0)

/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/biocrnpyler-1.0.2-py3.7.egg/biocrnpyler/chemical_reaction_network.py:243: UserWarning: simulate_with_bioscrape is depricated and will cease working in a future release. Instead, please use simulate_with_bioscrape_via_sbml.
/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/html5lib/_trie/_base.py:3: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping
/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/scipy/integrate/odepack.py:247: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run wi

In [5]:
#R

In [6]:
palette = colorcet.b_glasbey_category10
palette.remove('#d62628')

In [7]:
p = bokeh.plotting.figure(width = 450, height = 340, 
                         x_axis_label = 'time (sec)', y_axis_label = 'concentration',
                         title = 'ATP synthase tx and tl')
plot_list = ['rna_atpsyn', 'protein_atpsyn', 'dna_atpsyn']
for i,item in enumerate(plot_list):
    p.line(timepoints, R[item], legend_label = item, color = palette[i], line_width = 2)
    
p.legend.location = 'center_right'
bokeh.io.show(p)

## SBML 2 (simple) for ATP Synthase Tx and Tl
---

In [8]:
rna_atpsyn = Species("atpsyn", material_type = "rna")

dna_atpsyn = Species("atpsyn", material_type = "dna")

f_atpsyn = Species("atpsyn", material_type = "protein")

k_tx = 1
k_tl = 1

R = Reaction([dna_atpsyn],[rna_atpsyn], k_tx)

R1 = Reaction([rna_atpsyn], [f_atpsyn], k_tl)
CRN_simpletxtl = ChemicalReactionNetwork(species = [rna_atpsyn, dna_atpsyn, f_atpsyn], reactions = [R, R1])

print(CRN_simpletxtl.pretty_print(show_materials = True, show_rates = True, show_attributes = True))


TypeError: new_interface() missing 1 required positional argument: 'propensity_type'

In [ ]:
# Write SBML file
CRN_simpletxtl.write_sbml_file('atpsynthase_simple_txtl.xml')

In [ ]:
x0 = {'dna_atpsyn' : 10}
timepoints = np.linspace(0,100,100)
r_simple = CRN_simpletxtl.simulate_with_bioscrape(timepoints, initial_condition_dict = x0)

#results, _ = CRN_machinerytxtl.simulate_via_bioscrape(timepoints, initial_condition_dict = x0, sbml_warnings = False)

In [ ]:
p_simple = bokeh.plotting.figure(width = 450, height = 350, title = 'ATP Synthase with Machinery',
                          x_axis_label = 'time',
                          y_axis_label = 'concentration')
plot_list = [ 'dna_atpsyn']
#plot_str = ['atp', 'outside_H', 'inside_H', 'bound atp synthase', 'free atp synthase']
# plot_list = [atp_synthase_b_protein_id, atp_synthase_f_protein_id]
# plot_str = ['atp synthase bound', 'atp synthase free']
for i,item in enumerate(plot_list):
    p_simple.line(timepoints, r_simple[item], line_width = 2, legend_label = item, color = palette[i])
    
bokeh.io.show(p_simple)

## SBML 3 (manual RNA poly, Ribo, mrna degradation) for ATP Synthase Tx and Tl
---

In [ ]:
rna_atpsyn = Species("atpsyn", material_type = "rna")
dna_atpsyn = Species("atpsyn", material_type = "dna")
f_atpsyn = Species("atpsyn", material_type = "protein")

RNAP_machinery = Species('RNAP_machinery', material_type = 'protein')
Ribo_machinery = Species("Ribo_machinery", material_type = 'protein')
RNAse_machinery = Species("RNAse_machinery", material_type = 'protein')

complex_dna_rnap = ComplexSpecies([dna_atpsyn, RNAP_machinery])
complex_rna_ribo = ComplexSpecies([rna_atpsyn, Ribo_machinery])
complex_rnase_rna = ComplexSpecies([rna_atpsyn, RNAse_machinery])

kb = 50.0
kf = 1.0
k1 = 0.05
k5 = 0.001

R0 = Reaction([dna_atpsyn, RNAP_machinery], [complex_dna_rnap], k = kb) # THIS ONE WILL BE TEMP DEPENDENT, 
                                                                        # made irreversible for now
R1 = Reaction([complex_dna_rnap], [rna_atpsyn, RNAP_machinery], k = k1) #dna_atpsyn,
R2 = Reaction([rna_atpsyn, Ribo_machinery], [complex_rna_ribo], k = kb) #k_rev = kf)
R3 = Reaction([complex_rna_ribo],[rna_atpsyn, f_atpsyn, Ribo_machinery], k = k1)
R4 = Reaction([rna_atpsyn, RNAse_machinery], [complex_rnase_rna], k = kb)# k_rev = kf
R5 = Reaction([complex_rnase_rna], [RNAse_machinery], k = k5)

CRN_machinerytxtl = ChemicalReactionNetwork(species = [rna_atpsyn, dna_atpsyn, f_atpsyn, RNAP_machinery, Ribo_machinery,
                                         RNAse_machinery, complex_dna_rnap, complex_rna_ribo, complex_rnase_rna],
                               reactions = [R0, R1, R2, R3, R4, R5])

print(CRN_machinerytxtl.pretty_print(show_materials = True, show_rates = True, show_attributes = True))

In [ ]:
# Write SBML file
CRN_machinerytxtl.write_sbml_file('atpsynthase_machinery_txtl.xml')

In [ ]:
x0 = {'dna_atpsyn' : 10, 'protein_RNAP_machinery' : 10, 'protein_Ribo_machinery' : 10, 'protein_RNAse_machinery' : 10}
timepoints = np.linspace(0,1000,100)
r = CRN_machinerytxtl.simulate_with_bioscrape(timepoints, initial_condition_dict = x0)

#results, _ = CRN_machinerytxtl.simulate_via_bioscrape(timepoints, initial_condition_dict = x0, sbml_warnings = False)

In [ ]:
p_mach = bokeh.plotting.figure(width = 450, height = 350, title = 'ATP Synthase with Machinery',
                          x_axis_label = 'time',
                          y_axis_label = 'concentration')
plot_list = ['protein_atpsyn','dna_atpsyn', 'rna_atpsyn']
#plot_str = ['atp', 'outside_H', 'inside_H', 'bound atp synthase', 'free atp synthase']
# plot_list = [atp_synthase_b_protein_id, atp_synthase_f_protein_id]
# plot_str = ['atp synthase bound', 'atp synthase free']
for i,item in enumerate(plot_list):
    p_mach.line(timepoints, r[item], line_width = 2, legend_label = item, color = palette[i])
p_mach.legend.click_policy = 'hide'
p_mach.legend.location = 'top_left'
bokeh.io.show(p_mach)

In [ ]:
r.head()

## SBML for atp synthase temperature sensitive txtl 25 C
---

In [ ]:
rna_atpsyn = Species("atpsyn", material_type = "rna")
dna_atpsyn = Species("atpsyn", material_type = "dna")
f_atpsyn = Species("atpsyn", material_type = "protein")

RNAP_machinery = Species('RNAP_machinery', material_type = 'protein')
Ribo_machinery = Species("Ribo_machinery", material_type = 'protein')
RNAse_machinery = Species("RNAse_machinery", material_type = 'protein')

complex_dna_rnap = ComplexSpecies([dna_atpsyn, RNAP_machinery])
complex_rna_ribo = ComplexSpecies([rna_atpsyn, Ribo_machinery])
complex_rnase_rna = ComplexSpecies([rna_atpsyn, RNAse_machinery])

kb = 100.0
kf = 1.0
k1 = 0.05
k5 = 0.001
A = 0.0576
B = -4.68
C = 120
T = 25 

#rate_string= "(" + str(dna)+ "*"+ "(" + "1" + "/" + "(" + str(A) + "*" + str(Temperature) + "^" + "2" + "+" + str(B) + "*" + str(Temperature) + "+" + str(C) + ")"+ ")" + ")"

k_temp = 0.01 * (1/(A * T**2 + B * T + C))

R0 = Reaction([dna_atpsyn, RNAP_machinery], [complex_dna_rnap], k = k_temp) # THIS ONE WILL BE TEMP DEPENDENT, 
                                                                        # made irreversible for now
R1 = Reaction([complex_dna_rnap], [rna_atpsyn, RNAP_machinery], k = k1) #dna_atpsyn,
R2 = Reaction([rna_atpsyn, Ribo_machinery], [complex_rna_ribo], k = kb) #k_rev = kf)
R3 = Reaction([complex_rna_ribo],[rna_atpsyn, f_atpsyn, Ribo_machinery], k = k1)
R4 = Reaction([rna_atpsyn, RNAse_machinery], [complex_rnase_rna], k = kb)# k_rev = kf
R5 = Reaction([complex_rnase_rna], [RNAse_machinery], k = k5)

CRN_25temp_machinerytxtl_atpsyn = ChemicalReactionNetwork(species = [rna_atpsyn, dna_atpsyn, f_atpsyn, RNAP_machinery, Ribo_machinery,
                                         RNAse_machinery, complex_dna_rnap, complex_rna_ribo, complex_rnase_rna],
                               reactions = [R0, R1, R2, R3, R4, R5])

print(CRN_25temp_machinerytxtl_atpsyn.pretty_print(show_materials = True, show_rates = True, show_attributes = True))

In [ ]:
# Write SBML file
CRN_25temp_machinerytxtl_atpsyn.write_sbml_file('atpsyn_25temp_machinery_txtl.xml')

## SBML for atp synthase temperature sensitive txtl 37 C
---

In [ ]:
rna_atpsyn = Species("atpsyn", material_type = "rna")
dna_atpsyn = Species("atpsyn", material_type = "dna")
f_atpsyn = Species("atpsyn", material_type = "protein")

RNAP_machinery = Species('RNAP_machinery', material_type = 'protein')
Ribo_machinery = Species("Ribo_machinery", material_type = 'protein')
RNAse_machinery = Species("RNAse_machinery", material_type = 'protein')

complex_dna_rnap = ComplexSpecies([dna_atpsyn, RNAP_machinery])
complex_rna_ribo = ComplexSpecies([rna_atpsyn, Ribo_machinery])
complex_rnase_rna = ComplexSpecies([rna_atpsyn, RNAse_machinery])

kb = 100.0
kf = 1.0
k1 = 0.05
k5 = 0.001
A = 0.0576
B = -4.68
C = 120
T = 100

#rate_string= "(" + str(dna)+ "*"+ "(" + "1" + "/" + "(" + str(A) + "*" + str(Temperature) + "^" + "2" + "+" + str(B) + "*" + str(Temperature) + "+" + str(C) + ")"+ ")" + ")"

k_temp = 0.01 * (1/(A * T**2 + B * T + C))

R0 = Reaction([dna_atpsyn, RNAP_machinery], [complex_dna_rnap], k = k_temp) # THIS ONE WILL BE TEMP DEPENDENT, 
                                                                        # made irreversible for now
R1 = Reaction([complex_dna_rnap], [rna_atpsyn, RNAP_machinery], k = k1) #dna_atpsyn,
R2 = Reaction([rna_atpsyn, Ribo_machinery], [complex_rna_ribo], k = kb) #k_rev = kf)
R3 = Reaction([complex_rna_ribo],[rna_atpsyn, f_atpsyn, Ribo_machinery], k = k1)
R4 = Reaction([rna_atpsyn, RNAse_machinery], [complex_rnase_rna], k = kb)# k_rev = kf
R5 = Reaction([complex_rnase_rna], [RNAse_machinery], k = k5)

CRN_100temp_machinerytxtl_atpsyn = ChemicalReactionNetwork(species = [rna_atpsyn, dna_atpsyn, f_atpsyn, RNAP_machinery, Ribo_machinery,
                                         RNAse_machinery, complex_dna_rnap, complex_rna_ribo, complex_rnase_rna],
                               reactions = [R0, R1, R2, R3, R4, R5])

print(CRN_100temp_machinerytxtl_atpsyn.pretty_print(show_materials = True, show_rates = True, show_attributes = True))

In [ ]:
# Write SBML file
CRN_100temp_machinerytxtl_atpsyn.write_sbml_file('atpsyn_100temp_machinery_txtl.xml')

## SBML for ATP synthase membrane integration
---

In [ ]:
atpsyn = Species("atpsyn", material_type = "protein")

atpsyn_b = Species("atpsyn_b", material_type = "protein")

k_bound = 0.0003

R = Reaction([atpsyn], [atpsyn_b], k_bound)

#R1 = Reaction([atpsyn], [], k_bound)

CRN_mi = ChemicalReactionNetwork(species = [atpsyn, atpsyn_b], reactions = [R])

print(CRN_mi.pretty_print(show_materials = True, show_rates = True, show_attributes = True))

In [ ]:
# Write SBML file
CRN_mi.write_sbml_file('atpsynthase_membrane_integration.xml')

In [ ]:
timepoints = np.linspace(0,100,100)
x0_mi = {'protein_atpsyn':10}
R_mi = CRN_mi.simulate_with_bioscrape(timepoints, initial_condition_dict = x0_mi)

In [ ]:
R_mi

In [ ]:
p_mi = bokeh.plotting.figure(width = 450, height = 340, 
                         x_axis_label = 'time (sec)', y_axis_label = 'concentration',
                         title = 'ATP synthase membrane integration')
plot_list = ['protein_atpsyn', 'protein_atpsyn_b']
for i,item in enumerate(plot_list):
    p_mi.line(timepoints, R_mi[item], legend_label = item, color = palette[i], line_width = 2)
    
p_mi.legend.location = 'center_right'
bokeh.io.show(p_mi)

## SBML for ATP Synthesis via ATP synthase membrane protein
---

In [ ]:
outside_H = Species("outside_H", material_type = "proton")

inside_H = Species("inside_H", material_type = "proton")

atpsyn_b = Species("atpsyn_b", material_type = "protein")

adp = Species('adp', material_type = 'metabolite')
atp = Species('atp', material_type = 'metabolite')
pi = Species('pi', material_type = 'metabolite')


k_atp_synthesis = 100

R1 = Reaction([outside_H, outside_H, outside_H, outside_H, atpsyn_b], 
              [inside_H, inside_H, inside_H, inside_H, atpsyn_b], k_atp_synthesis)

R3 = Reaction([inside_H, inside_H, inside_H, inside_H, atpsyn_b],
              [outside_H, outside_H, outside_H, outside_H, atpsyn_b], k_atp_synthesis)

R2 = Reaction([adp, pi], [atp], k_atp_synthesis)

# R2 = Reaction([outside_H, outside_H, outside_H, outside_H], 
#              [inside_H, inside_H, inside_H, inside_H], k_atp_synthesis)

CRN_atpsynthesis = ChemicalReactionNetwork(species = [outside_H, inside_H, atpsyn_b, adp, atp, pi], reactions = [R1, R2,R3])

print(CRN_atpsynthesis.pretty_print(show_materials = True, show_rates = True, show_attributes = True))

In [ ]:
# Write SBML file
CRN_atpsynthesis.write_sbml_file('atp_synthesis_through_atpsynthase_try2.xml')

In [ ]:
timepoints = np.linspace(0,2,100)
x0_atpsynthesis = {'proton_outside_H':10, 'metabolite_adp':10, 'metabolite_pi':10}
R_atpsynthesis = CRN_atpsynthesis.simulate_with_bioscrape(timepoints, initial_condition_dict = x0_atpsynthesis)

In [ ]:
p_atpsynthesis = bokeh.plotting.figure(width = 450, height = 340, 
                         x_axis_label = 'time (sec)', y_axis_label = 'concentration',
                         title = 'ATP synthesis through ATP synthase')
plot_list = ['proton_outside_H', 'proton_inside_H', 'metabolite_atp']
for i,item in enumerate(plot_list):
    p_atpsynthesis.line(timepoints, R_atpsynthesis[item], legend_label = item, color = palette[i], line_width = 2)
    
p_atpsynthesis.legend.location = 'center_right'
bokeh.io.show(p_atpsynthesis)

## SBML for Proton Pump transcription and translation
---

In [ ]:
G_pp = DNAassembly("proton_pump", promoter = "atpsyn_proton_pump", rbs = "proton_pumnp_rbs", transcript = None, protein = None)
M3_pp = TxTlDilutionMixture("liposome", components = [G_pp], parameter_file = "default_parameters.txt")
CRN3_pp = M3_pp.compile_crn()

#print(repr(M3),"\n", CRN3_pp.pretty_print(show_attributes = True, show_material = True, show_rates = True),"\n\n")



In [ ]:
# Write SBML file
CRN3_pp.write_sbml_file('proton_pump_tx_tl.xml')

In [ ]:
timepoints = np.linspace(0,10000,100)
x0_pp = {'dna_proton_pump':10, 'protein_RNAP_machinery':10, 'protein_Ribo_machinery':10,
     'protein_RNAase_machinery':10}
R_pp = CRN3_pp.simulate_with_bioscrape(timepoints, initial_condition_dict = x0_pp)

In [ ]:
p_pp = bokeh.plotting.figure(width = 450, height = 340, 
                         x_axis_label = 'time (sec)', y_axis_label = 'concentration',
                         title = 'Proton pump tx and tl')
plot_list = ['rna_proton_pump', 'protein_proton_pump', 'dna_proton_pump']
for i,item in enumerate(plot_list):
    p_pp.line(timepoints, R_pp[item], legend_label = item, color = palette[i], line_width = 2)
    
p_pp.legend.location = 'center_right'
bokeh.io.show(p_pp)

## SBML for proton pump Simple tx tl
---

In [ ]:
rna_proton_pump = Species("proton_pump", material_type = "rna")

dna_proton_pump = Species("proton_pump", material_type = "dna")

f_proton_pump = Species("proton_pump", material_type = "protein")

k_tx = 1
k_tl = 1

R = Reaction([dna_proton_pump],[rna_proton_pump], k_tx)

R1 = Reaction([rna_proton_pump], [f_proton_pump], k_tl)
CRN_simpletxtl_pp = ChemicalReactionNetwork(species = [rna_proton_pump, dna_proton_pump, f_proton_pump], 
                                         reactions = [R, R1])

print(CRN_simpletxtl_pp.pretty_print(show_materials = True, show_rates = True, show_attributes = True))



In [ ]:
# Write SBML file
CRN_simpletxtl_pp.write_sbml_file('proton_pump_simple_txtl.xml')

## SBML for proton pump machinery txtl
---

In [ ]:
rna_proton_pump = Species("proton_pump", material_type = "rna")
dna_proton_pump = Species("proton_pump", material_type = "dna")
f_proton_pump = Species("proton_pump", material_type = "protein")

RNAP_machinery = Species('RNAP_machinery', material_type = 'protein')
Ribo_machinery = Species("Ribo_machinery", material_type = 'protein')
RNAse_machinery = Species("RNAse_machinery", material_type = 'protein')

complex_dna_rnap = ComplexSpecies([dna_proton_pump, RNAP_machinery])
complex_rna_ribo = ComplexSpecies([rna_proton_pump, Ribo_machinery])
complex_rnase_rna = ComplexSpecies([rna_proton_pump, RNAse_machinery])

kb = 100.0
kf = 1.0
k1 = 0.05
k5 = 0.001
# A = 0.0576
# B = -4.68
# C = 120
# T = 25 

# #rate_string= "(" + str(dna)+ "*"+ "(" + "1" + "/" + "(" + str(A) + "*" + str(Temperature) + "^" + "2" + "+" + str(B) + "*" + str(Temperature) + "+" + str(C) + ")"+ ")" + ")"

# k_temp = 0.01 * (1/(A * T**2 + B * T + C))

R0 = Reaction([dna_proton_pump, RNAP_machinery], [complex_dna_rnap], k = kb) # THIS ONE WILL BE TEMP DEPENDENT, 
                                                                        # made irreversible for now
R1 = Reaction([complex_dna_rnap], [rna_proton_pump, RNAP_machinery], k = k1) #dna_atpsyn,
R2 = Reaction([rna_proton_pump, Ribo_machinery], [complex_rna_ribo], k = kb) #k_rev = kf)
R3 = Reaction([complex_rna_ribo],[rna_proton_pump, f_proton_pump, Ribo_machinery], k = k1)
R4 = Reaction([rna_proton_pump, RNAse_machinery], [complex_rnase_rna], k = kb)# k_rev = kf
R5 = Reaction([complex_rnase_rna], [RNAse_machinery], k = k5)

CRN_machinerytxtl_proton_pump = ChemicalReactionNetwork(species = [rna_proton_pump, dna_proton_pump, f_proton_pump, RNAP_machinery, Ribo_machinery,
                                         RNAse_machinery, complex_dna_rnap, complex_rna_ribo, complex_rnase_rna],
                               reactions = [R0, R1, R2, R3, R4, R5])

print(CRN_machinerytxtl_proton_pump.pretty_print(show_materials = True, show_rates = True, show_attributes = True))

In [ ]:
# Write SBML file
CRN_machinerytxtl_proton_pump.write_sbml_file('proton_pump_machinery_txtl.xml')

## SBML for proton pump temperature sensitive txtl 25 C
---

In [ ]:
rna_proton_pump = Species("proton_pump", material_type = "rna")
dna_proton_pump = Species("proton_pump", material_type = "dna")
f_proton_pump = Species("proton_pump", material_type = "protein")

RNAP_machinery = Species('RNAP_machinery', material_type = 'protein')
Ribo_machinery = Species("Ribo_machinery", material_type = 'protein')
RNAse_machinery = Species("RNAse_machinery", material_type = 'protein')

complex_dna_rnap = ComplexSpecies([dna_proton_pump, RNAP_machinery])
complex_rna_ribo = ComplexSpecies([rna_proton_pump, Ribo_machinery])
complex_rnase_rna = ComplexSpecies([rna_proton_pump, RNAse_machinery])

kb = 100.0
kf = 1.0
k1 = 0.05
k5 = 0.001
A = 0.0576
B = -4.68
C = 120
T = 25 

#rate_string= "(" + str(dna)+ "*"+ "(" + "1" + "/" + "(" + str(A) + "*" + str(Temperature) + "^" + "2" + "+" + str(B) + "*" + str(Temperature) + "+" + str(C) + ")"+ ")" + ")"

k_temp = 0.01 * (1/(A * T**2 + B * T + C))

R0 = Reaction([dna_proton_pump, RNAP_machinery], [complex_dna_rnap], k = k_temp) # THIS ONE WILL BE TEMP DEPENDENT, 
                                                                        # made irreversible for now
R1 = Reaction([complex_dna_rnap], [rna_proton_pump, RNAP_machinery], k = k1) #dna_atpsyn,
R2 = Reaction([rna_proton_pump, Ribo_machinery], [complex_rna_ribo], k = kb) #k_rev = kf)
R3 = Reaction([complex_rna_ribo],[rna_proton_pump, f_proton_pump, Ribo_machinery], k = k1)
R4 = Reaction([rna_proton_pump, RNAse_machinery], [complex_rnase_rna], k = kb)# k_rev = kf
R5 = Reaction([complex_rnase_rna], [RNAse_machinery], k = k5)

CRN_25temp_machinerytxtl_proton_pump = ChemicalReactionNetwork(species = [rna_proton_pump, dna_proton_pump, f_proton_pump, RNAP_machinery, Ribo_machinery,
                                         RNAse_machinery, complex_dna_rnap, complex_rna_ribo, complex_rnase_rna],
                               reactions = [R0, R1, R2, R3, R4, R5])

print(CRN_25temp_machinerytxtl_proton_pump.pretty_print(show_materials = True, show_rates = True, show_attributes = True))

In [ ]:
# Write SBML file
CRN_25temp_machinerytxtl_proton_pump.write_sbml_file('proton_pump_25temp_machinery_txtl.xml')

## SBML for proton pump temperature sensitive txtl 37 C
---

In [ ]:
rna_proton_pump = Species("proton_pump", material_type = "rna")
dna_proton_pump = Species("proton_pump", material_type = "dna")
f_proton_pump = Species("proton_pump", material_type = "protein")

RNAP_machinery = Species('RNAP_machinery', material_type = 'protein')
Ribo_machinery = Species("Ribo_machinery", material_type = 'protein')
RNAse_machinery = Species("RNAse_machinery", material_type = 'protein')

complex_dna_rnap = ComplexSpecies([dna_proton_pump, RNAP_machinery])
complex_rna_ribo = ComplexSpecies([rna_proton_pump, Ribo_machinery])
complex_rnase_rna = ComplexSpecies([rna_proton_pump, RNAse_machinery])

kb = 100.0
kf = 1.0
k1 = 0.05
k5 = 0.001
A = 0.0576
B = -4.68
C = 120
T = 100

#rate_string= "(" + str(dna)+ "*"+ "(" + "1" + "/" + "(" + str(A) + "*" + str(Temperature) + "^" + "2" + "+" + str(B) + "*" + str(Temperature) + "+" + str(C) + ")"+ ")" + ")"

k_temp = 0.01 * (1/(A * T**2 + B * T + C))

R0 = Reaction([dna_proton_pump, RNAP_machinery], [complex_dna_rnap], k = k_temp) # THIS ONE WILL BE TEMP DEPENDENT, 
                                                                        # made irreversible for now
R1 = Reaction([complex_dna_rnap], [rna_proton_pump, RNAP_machinery], k = k1) #dna_atpsyn,
R2 = Reaction([rna_proton_pump, Ribo_machinery], [complex_rna_ribo], k = kb) #k_rev = kf)
R3 = Reaction([complex_rna_ribo],[rna_proton_pump, f_proton_pump, Ribo_machinery], k = k1)
R4 = Reaction([rna_proton_pump, RNAse_machinery], [complex_rnase_rna], k = kb)# k_rev = kf
R5 = Reaction([complex_rnase_rna], [RNAse_machinery], k = k5)

CRN_100temp_machinerytxtl_proton_pump = ChemicalReactionNetwork(species = [rna_proton_pump, dna_proton_pump, f_proton_pump, RNAP_machinery, Ribo_machinery,
                                         RNAse_machinery, complex_dna_rnap, complex_rna_ribo, complex_rnase_rna],
                               reactions = [R0, R1, R2, R3, R4, R5])

print(CRN_100temp_machinerytxtl_proton_pump.pretty_print(show_materials = True, show_rates = True, show_attributes = True))

In [ ]:
# Write SBML file
CRN_100temp_machinerytxtl_proton_pump.write_sbml_file('proton_pump_100temp_machinery_txtl.xml')

## SBML for proton pump membrane integration
---

In [ ]:
proton_pump = Species("proton_pump", material_type = "protein")

proton_pump_b = Species("proton_pump_b", material_type = "protein")

k_bound = 0.0003

R_pp = Reaction([proton_pump], [proton_pump_b], k_bound)

CRN_mi_pp = ChemicalReactionNetwork(species = [proton_pump, proton_pump_b], reactions = [R_pp])

print(CRN_mi_pp.pretty_print(show_materials = True, show_rates = True, show_attributes = True))

In [ ]:
# Write SBML file
CRN_mi_pp.write_sbml_file('proton_pump_membrane_integration.xml')

In [ ]:

outside_H = Species("outside_H", material_type = "proton")

inside_H = Species("inside_H", material_type = "proton")

adp = Species('adp', material_type = 'metabolite')
atp = Species('atp', material_type = 'metabolite')
pi = Species('pi', material_type = 'metabolite')


k_atp_synthesis = 2

R1 = Reaction([adp,pi], [atp], k_atp_synthesis)

R2 = Reaction([outside_H, outside_H, outside_H, outside_H], 
             [inside_H, inside_H, inside_H, inside_H], k_atp_synthesis)

CRN_atpsynthesis = ChemicalReactionNetwork(species = [outside_H, inside_H, adp, atp, pi], reactions = [R1,R2])

print(CRN_atpsynthesis.pretty_print(show_materials = True, show_rates = True, show_attributes = True))

In [ ]:
timepoints = np.linspace(0,10,100)
x0_mi_pp = {'protein_proton_pump':10}
R_mi_pp = CRN_mi_pp.simulate_with_bioscrape(timepoints, initial_condition_dict = x0_mi_pp)

In [ ]:
p_mi_pp = bokeh.plotting.figure(width = 450, height = 340, 
                         x_axis_label = 'time (sec)', y_axis_label = 'concentration',
                         title = 'Proton pump membrane integration')
plot_list = ['protein_proton_pump', 'protein_proton_pump_b']
for i,item in enumerate(plot_list):
    p_mi_pp.line(timepoints, R_mi_pp[item], legend_label = item, color = palette[i], line_width = 2)
    
p_mi_pp.legend.location = 'center_right'
bokeh.io.show(p_mi_pp)

## SBML for proton transport through proton pump
---

In [ ]:
outside_H = Species("outside_H", material_type = "proton")

inside_H = Species("inside_H", material_type = "proton")

adp = Species('adp', material_type = 'metabolite')
atp = Species('atp', material_type = 'metabolite')
pi = Species('pi', material_type = 'metabolite')


k_proton_out = 100
k_proton_atp = 0.2

#R1 = Reaction([adp,pi], [atp], k_proton_atp)
R1 = Reaction([atp], [adp, pi], k_proton_atp)


R2 = Reaction([inside_H, inside_H, inside_H, inside_H],
              [outside_H, outside_H, outside_H, outside_H], k_proton_out)

CRN_protontransport = ChemicalReactionNetwork(species = [outside_H, inside_H, adp, atp, pi], reactions = [R1, R2])

print(CRN_protontransport.pretty_print(show_materials = True, show_rates = True, show_attributes = True))

In [ ]:
# Write SBML file
CRN_protontransport.write_sbml_file('proton_transport_through_proton_pump.xml')

In [ ]:
timepoints = np.linspace(0,2,100)
x0_protontransport = {'proton_inside_H':10, 'metabolite_adp':10, 'metabolite_pi':10}
R_protontransport = CRN_protontransport.simulate_with_bioscrape(timepoints, 
                                                                initial_condition_dict = x0_protontransport)


In [ ]:
p_protontransport = bokeh.plotting.figure(width = 450, height = 340, 
                         x_axis_label = 'time (sec)', y_axis_label = 'concentration',
                         title = 'Proton transport through proton pump')
plot_list = ['proton_outside_H', 'proton_inside_H', 'metabolite_atp']
for i,item in enumerate(plot_list):
    p_protontransport.line(timepoints, R_protontransport[item], legend_label = item, color = palette[i], line_width = 2)
    
p_protontransport.legend.location = 'center_right'
bokeh.io.show(p_protontransport)

## SBML for ATP Use
---

In [ ]:
adp = Species('adp', material_type = 'metabolite')
atp = Species('atp', material_type = 'metabolite')
pi = Species('pi', material_type = 'metabolite')

k_atp_use = 75
# used to be 5

R1_atpuse = Reaction([atp], [adp, pi], k_atp_use)


CRN_atpuse = ChemicalReactionNetwork(species = [adp, atp, pi], reactions = [R1_atpuse])

print(CRN_atpuse.pretty_print(show_materials = True, show_rates = True, show_attributes = True))

In [ ]:
# Write SBML file
CRN_atpuse.write_sbml_file('atp_75_use.xml')

In [ ]:
timepoints = np.linspace(0,75000,100)
plt_timepoints = timepoints/60/60
x0_atpuse = {'metabolite_atp':10}
R_atpuse = CRN_atpuse.simulate_with_bioscrape(timepoints, initial_condition_dict = x0_atpuse)

In [ ]:
p_atpuse = bokeh.plotting.figure(width = 450, height = 340, 
                         x_axis_label = 'time (sec)', y_axis_label = 'concentration',
                         title = 'ATP Use')
plot_list = ['metabolite_atp']
for i,item in enumerate(plot_list):
    p_atpuse.line(plt_timepoints, R_atpuse[item], legend_label = item, color = palette[i], line_width = 2)
    
p_atpuse.legend.location = 'center_right'
bokeh.io.show(p_atpuse)